In [36]:
import os
import re
import docx
import PyPDF2
import io 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import pandas as pd

[nltk_data] Downloading package stopwords to C:\Users\Moin
[nltk_data]     Dalvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Moin
[nltk_data]     Dalvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import click._bashcomplete

In [1]:
import aspose.words as aw
output = aw.Document()

In [37]:
# Function to extract text from resume
def getText(filename):
      
    # Create empty string 
    fullText = ''
    if filename.endswith('.docx'):
        doc = docx.Document(filename)
        
        for para in doc.paragraphs:
            fullText = fullText + para.text
            
           
    elif filename.endswith('.pdf'):  
        with open(filename, "rb") as pdf_file:
            pdoc = PyPDF2.PdfFileReader(filename)
            number_of_pages = pdoc.getNumPages()
            page = pdoc.pages[0]
            page_content = page.extractText()
             
        for paragraph in page_content:
            fullText =  fullText + paragraph
            
    else:
        import aspose.words as aw
        output = aw.Document()
        # Remove all content from the destination document before appending.
        output.remove_all_children()
        input = aw.Document(filename)
        # Append the source document to the end of the destination document.
        output.append_document(input, aw.ImportFormatMode.KEEP_SOURCE_FORMATTING)
        output.save("Output.docx");
        doc = docx.Document('Output.docx')
        
        for para in doc.paragraphs:
            fullText = fullText + para.text
        fullText = fullText[79:]
         
    return (fullText)

In [38]:
# Function to remove punctuation and tokenize the text
def tokenText(extText):
   
    # Remove punctuation marks
    punc = '''!()-[]{};:'"\,.<>/?@#$%^&*_~'''
    for ele in extText:
        if ele in punc:
            puncText = extText.replace(ele, "")
            
    # Tokenize the text and remove stop words
    stop_words = set(stopwords.words('english'))
    puncText.split()
    word_tokens = word_tokenize(puncText)
    TokenizedText = [w for w in word_tokens if not w.lower() in stop_words]
    TokenizedText = []
  
    for w in word_tokens:
        if w not in stop_words:
            TokenizedText.append(w)
    return(TokenizedText)

In [39]:
# Define key terms dictionary for fixing Role Applied for 
terms = {'WorkDay ERP':['workday', 'workday consultant', 'workday hcm', 'eib', 'picof', 
                        'workday studio','nnbound/outbound integrations'],
         'Peoplesoft':['peoplesoft', 'pia','ccb','birt','peci','ccw','pum','people tools',
                        'peoplesoft implementation','peoplesoft components',
                        'peoplesoft dba','peoplesoft admin','peoplesoft admin/dba','peopleSoft fscm', 
                        'peopletoolsupgrade','peopletools upgrade','process scheduler servers',
                        'peoplesoft hrms','peopleSoft consultant','peopledoft cloud',
                        'PeopleSoft migrations','eoplesoft Testing Framework','pure internet architecture'],             
         'Database Developer':['sql','sql server', 'ms sql server','msbi', 'sql developer', 'ssis','ssrs',
                        'ssms','t-sql','tsql','Razorsql', 'razor sql','triggers','powerbi','power bi',
                        'oracle sql', 'pl/sql', 'pl\sql','oracle', 'oracle 11g','oledb','cte','ddl',
                        'dml','etl','mariadb','maria db'],
         'Java Developer':['reactjs', 'react js', 'react js developer', 'html', 
                        'css3','xml','javascript','html5','boostrap','jquery', 'redux','php', 'node js',
                        'nodejs','apache','netbeans','nestjs','nest js','react developer','react hooks',
                        'jenkins']}

In [40]:
# List of all key terms to indicate skillset. Include all the key words in the list 
allTerms = ['workday', 'hcm', 'eib', 'picof','workday hcm',
                        'workday studio','nnbound/outbound integrations',
                        'peoplesoft', 'pia','ccb','birt','peci','ccw','pum','people tools',
                        'peoplesoft implementation','peoplesoft components',
                        'peoplesoft dba','peoplesoft admin','peoplesoft admin/dba','peopleSoft fscm', 
                        'peopletoolsupgrade','peopletools upgrade','process scheduler servers',
                        'peoplesoft hrms','peopleSoft consultant','peopledoft cloud',
                        'PeopleSoft migrations','eoplesoft Testing Framework','pure internet architecture',
                        'sql','sql server', 'ms sql server','msbi', 'sql developer', 'ssis','ssrs',
                        'ssms','t-sql','tsql','Razorsql', 'razor sql','triggers','powerbi','power bi',
                        'oracle sql', 'pl/sql', 'pl\sql','oracle', 'oracle 11g','oledb','cte','ddl',
                        'dml','etl','mariadb','maria db','reactjs', 'react js', 'react js developer', 'html', 
                        'css3','xml','javascript','html5','boostrap','jquery', 'redux','php', 'node js',
                        'nodejs','apache','netbeans','nestjs','nest js','react developer','react hooks',
                        'jenkins']

In [41]:
# Function to extract Name and contact details
def contactDetails(Text):
    name = ''  
    for i in range(0,3):
        name = " ".join([name, Text[i]])
    return(name)

In [42]:
# Function to extract experience details
def expDetails(Text):
    global sent
   
    Text = Text.split()
   
    for i in range(len(Text)-2):
        Text[i].lower()
        
        if Text[i] ==  'years':
            sent =  Text[i-2] + ' ' + Text[i-1] +' ' + Text[i] +' '+ Text[i+1] +' ' + Text[i+2]
            l = re.findall('\d*\.?\d+',sent)
            for i in l:
                a = float(i)
            return(a)
            return (sent)

In [43]:
# Function to extract skill set details
def skillSet(Text):
    t = []
    for i in range(len(Text)):
        if Text[i] in allTerms:
            if Text[i] in t:
                continue
            t.append(Text[i]) 
    return(t)

In [44]:
# Create an empt Data Frame ResumeText with two columns
ResumeText = pd.DataFrame([], columns=['Name', 'Exp_years', 'SkillSet','TextInfo'])

# Mention the path in your computer where resumes folder is stored
path = 'C:/Users\Moin Dalvi\Documents\Data Science Material\Projects\Resume Classification/Resumes'
text =[]
role = []

# Search the directory path and loop through the resume documents and callthe function getText
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        extText = getText(file_path)
        #print(type(extText))
        tokText = tokenText(extText)
        #print(extText)
        role.append(folder)
        Name = contactDetails(tokText)
        experience = expDetails(extText)
        skills = skillSet(tokText)
        NewRow = [Name,experience, skills,tokText]  
        ResumeText.loc[len(ResumeText)] = NewRow
ResumeText['RoleApplied'] = role

In [34]:
ResumeText

,Name,Exp_years,SkillSet,TextInfo,RoleApplied
0,Name Ravali P,NaN,[],"[Name, Ravali, P, Curriculum, Vitae, Specializ...",Internship
1,SUSOVAN BAG Seeking,NaN,[],"[SUSOVAN, BAG, Seeking, challenging, position,...",Internship
2,Anubhav Kumar Singh,NaN,[],"[Anubhav, Kumar, Singh, To, work, globally, co...",Peoplesoft Admin
3,Evaluation Only Created,7.0,[],"[Evaluation, Only, Created, AsposeWords, Copyr...",Peoplesoft Admin
4,Evaluation Only Created,4.0,[],"[Evaluation, Only, Created, AsposeWords, Copyr...",Peoplesoft Admin
...,...,...,...,...,...
74,Evaluation Only Created,4.0,[workday],"[Evaluation, Only, Created, AsposeWords, Copyr...",Workday
75,Seeking suitable positions,4.0,[workday],"[Seeking, suitable, positions, Workday, HCM, T...",Workday
76,WORKDAY | HCM,6.0,[workday],"[WORKDAY, |, HCM, |, FCMName, :, Kumar, SSRole...",Workday
77,Evaluation Only .,5.3,[],"[Evaluation, Only, ., Created, Aspose.Words, ....",Workday


In [16]:
ResumeText.head(50)

,Name,Exp_years,SkillSet,TextInfo,RoleApplied
0,Name Ravali P,NaN,[],"[Name, Ravali, P, Curriculum, Vitae, Specializ...",Internship
1,SUSOVAN BAG Seeking,NaN,[],"[SUSOVAN, BAG, Seeking, challenging, position,...",Internship
2,Anubhav Kumar Singh,NaN,[],"[Anubhav, Kumar, Singh, To, work, globally, co...",Peoplesoft Admin
3,Profile Summary :,7.00,[],"[Profile, Summary, :, 7+, years, experience, i...",Peoplesoft Admin
4,PeopleSoft Database Administrator,4.00,[],"[PeopleSoft, Database, Administrator, Gangared...",Peoplesoft Admin
5,MuraliExperience Summary I,6.00,[],"[MuraliExperience, Summary, I, 6, years, exper...",Peoplesoft Admin
6,"Priyanka Ramadoss61/46 ,",2016.00,[],"[Priyanka, Ramadoss61/46, ,, MountPleasant, ,,...",Peoplesoft Admin
7,PROFILE SUMMARYI overall,NaN,[],"[PROFILE, SUMMARYI, overall, 6.8, years, ’, ex...",Peoplesoft Admin
8,PEOPLESOFT AdministratorSRINIVASK Experience,10.00,[],"[PEOPLESOFT, AdministratorSRINIVASK, Experienc...",Peoplesoft Admin
9,PeopleSoft AdminVARKALA VIKASCareer,4.20,[],"[PeopleSoft, AdminVARKALA, VIKASCareer, Object...",Peoplesoft Admin


In [45]:
ResumeText.TextInfo[40]

['Page',
 '|',
 '1',
 'Name',
 ':',
 'M',
 'Prabakaran',
 'Title',
 ':',
 'UI',
 'Developer',
 'PROFESSIONAL',
 'SUMMARY',
 '●',
 '24+',
 'years',
 'Professional',
 'IT',
 'experience',
 'software',
 'developer',
 'knowledge',
 'different',
 'UI',
 'based',
 'Application',
 '●',
 'Hands',
 'experience',
 'HTML',
 ',',
 'CSS',
 ',',
 'JS',
 ',',
 'ReactJS',
 '●',
 'Hands',
 'experience',
 'handling',
 'UI',
 'interaction',
 ',',
 'Design',
 'methodology',
 '●',
 'Handling',
 'In',
 '-App',
 'purchase',
 ',',
 'uploading',
 'maintaining',
 'apps',
 'play',
 'store',
 '●',
 'Hands',
 'experience',
 'customization',
 'base',
 '-product',
 'depends',
 'client',
 'requirement',
 '●',
 'Cohesive',
 'team',
 'worker',
 ',',
 'strong',
 'analytical',
 ',',
 'problem',
 'solving',
 'interpersonal',
 'skills',
 'EDUCATION',
 '●',
 'Completed',
 '2017',
 'Bachelor',
 'Technology',
 '(',
 'ECE',
 ')',
 ',',
 'PRIST',
 'University',
 ',',
 'Tamil',
 'Nadu',
 '●',
 'Completed',
 '2012',
 'Higher',
 '

In [17]:
ResumeText.to_csv('Details_Extracted.csv', index = False)